# 1. Cài đặt các thư viện cần thiết

In [1]:
# Cài đặt thư viện
%pip install -q kaggle holidays meteostat

Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
import requests
import pandas as pd
from datetime import datetime
import holidays
from pyspark.sql import SparkSession
from pyspark.sql.functions import min, max

try:
    # Cố gắng lấy session hiện tại hoặc tạo mới
    spark = SparkSession.builder.getOrCreate()
    print("✅ Spark Session đã sẵn sàng!")
except Exception as e:
    print(f"❌ Không thể khởi tạo Spark: {e}")

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
26/01/09 03:08:37 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


✅ Spark Session đã sẵn sàng!


# 2. Khai báo đường dẫn lưu trữ trong Lakehouse

In [3]:
import os

# Đường dẫn đến thư mục Files trong Lakehouse (OneLake)
base_path = "/lakehouse/default/Files/bronze"

# Tạo các thư mục con để quản lý gọn gàng
os.makedirs(f"{base_path}/taxi_trips", exist_ok=True)
os.makedirs(f"{base_path}/taxi_zones", exist_ok=True)
os.makedirs(f"{base_path}/weather", exist_ok=True)
os.makedirs(f"{base_path}/holidays", exist_ok=True)

print(f"✅ Đã tạo thư mục tại: {base_path}")

✅ Đã tạo thư mục tại: /lakehouse/default/Files/bronze


# 3. Dataset_1: Tải Dataset chính (NYC Yellow Taxi)

In [5]:
import os
import requests
from datetime import datetime

# 1. CẤU HÌNH THỜI GIAN
START_YEAR = 2021
CURRENT_YEAR = datetime.now().year
CURRENT_MONTH = datetime.now().month

# Đường dẫn lưu trữ
base_path = "/lakehouse/default/Files/bronze/taxi_trips"
os.makedirs(base_path, exist_ok=True)

print(f"🚀 Bắt đầu tải dữ liệu từ {START_YEAR} đến nay...")
print(f"📂 Lưu tại: {base_path}")

# 2. VÒNG LẶP THEO NĂM VÀ THÁNG
for year in range(START_YEAR, CURRENT_YEAR + 1):
    for month in range(1, 13):
        # Dừng nếu vượt quá thời điểm hiện tại (trừ hao 2 tháng vì dữ liệu thường chậm hơn thực tế)
        if year == CURRENT_YEAR and month > CURRENT_MONTH:
            break
            
        # Định dạng tháng thành 2 chữ số (vd: 1 -> 01)
        month_str = f"{month:02d}"
        file_name = f"yellow_tripdata_{year}-{month_str}.parquet"
        save_path = f"{base_path}/{file_name}"
        
        # URL chuẩn của NYC TLC (đã chuyển đổi dữ liệu cũ sang Parquet)
        url = f"https://d37ci6vzurychx.cloudfront.net/trip-data/{file_name}"

        # Kiểm tra file đã tồn tại chưa để tránh tải lại
        if os.path.exists(save_path):
            print(f"⏩ [Bỏ qua] {file_name} (Đã tồn tại)")
            continue

        # Tải file
        try:
            # Timeout 30s để tránh treo nếu mạng lag
            response = requests.get(url, stream=True, timeout=30) 
            
            if response.status_code == 200:
                with open(save_path, 'wb') as f:
                    for chunk in response.iter_content(chunk_size=1024*1024): # Tải từng chunk 1MB
                        if chunk:
                            f.write(chunk)
                print(f"✅ [Tải xong] {file_name}")
            elif response.status_code == 403 or response.status_code == 404:
                # Dữ liệu Taxi thường bị trễ 2-3 tháng, lỗi 403/404 là bình thường với các tháng gần nhất
                print(f"⚠️ [Không tìm thấy] {file_name} (Có thể chưa phát hành)")
            else:
                print(f"❌ [Lỗi {response.status_code}] {file_name}")
                
        except Exception as e:
            print(f"❌ [Lỗi kết nối] {file_name}: {e}")

print("🎉 Hoàn tất quy trình tải dữ liệu!")

🚀 Bắt đầu tải dữ liệu từ 2021 đến nay...
📂 Lưu tại: /lakehouse/default/Files/bronze/taxi_trips
✅ [Tải xong] yellow_tripdata_2021-01.parquet
✅ [Tải xong] yellow_tripdata_2021-02.parquet
✅ [Tải xong] yellow_tripdata_2021-03.parquet
✅ [Tải xong] yellow_tripdata_2021-04.parquet
✅ [Tải xong] yellow_tripdata_2021-05.parquet
✅ [Tải xong] yellow_tripdata_2021-06.parquet
✅ [Tải xong] yellow_tripdata_2021-07.parquet
✅ [Tải xong] yellow_tripdata_2021-08.parquet
✅ [Tải xong] yellow_tripdata_2021-09.parquet
✅ [Tải xong] yellow_tripdata_2021-10.parquet
✅ [Tải xong] yellow_tripdata_2021-11.parquet
✅ [Tải xong] yellow_tripdata_2021-12.parquet
✅ [Tải xong] yellow_tripdata_2022-01.parquet
✅ [Tải xong] yellow_tripdata_2022-02.parquet
✅ [Tải xong] yellow_tripdata_2022-03.parquet
✅ [Tải xong] yellow_tripdata_2022-04.parquet
✅ [Tải xong] yellow_tripdata_2022-05.parquet
✅ [Tải xong] yellow_tripdata_2022-06.parquet
✅ [Tải xong] yellow_tripdata_2022-07.parquet
✅ [Tải xong] yellow_tripdata_2022-08.parquet
✅ [Tả

In [6]:
import pandas as pd
# Load data into pandas DataFrame from "/lakehouse/default/Files/bronze/taxi_trips/yellow_tripdata_2025-10.parquet"
df = pd.read_parquet("/lakehouse/default/Files/bronze/taxi_trips/yellow_tripdata_2025-10.parquet")
display(df)

# 4. Dataset_2: Tải NYC Taxi taxi_zones (Lookup Table)

In [7]:
# ==========================================
# 1. CẤU HÌNH THỜI GIAN ĐỒNG BỘ
# ==========================================
# Phải khớp với khoảng thời gian bạn tải Taxi Data
START_YEAR = 2021
CURRENT_YEAR = datetime.now().year 

# Đường dẫn gốc
base_path = "/lakehouse/default/Files/bronze"
os.makedirs(f"{base_path}/taxi_zones", exist_ok=True)
print(f"🔄 Đang đồng bộ dữ liệu phụ trợ từ năm {START_YEAR} đến {CURRENT_YEAR}...")

# ==========================================
# 2. DATASET 2: NYC TAXI taxi_zones (Lookup Table)
# ==========================================
# Đây là dữ liệu TĨNH (Static), không thay đổi theo năm nên chỉ cần tải 1 lần.
print("\n[2/4] 🚏 Đang tải Taxi taxi_zones (Bản đồ khu vực)...")
url_taxi_zones = "https://d37ci6vzurychx.cloudfront.net/misc/taxi+_zone_lookup.csv"
save_path_taxi_zones = f"{base_path}/taxi_zones/taxi_zone_lookup.csv"

if not os.path.exists(save_path_taxi_zones):
    try:
        r = requests.get(url_taxi_zones)
        with open(save_path_taxi_zones, 'wb') as f:
            f.write(r.content)
        print("✅ Đã tải xong Taxi taxi_zones.")
    except Exception as e:
        print(f"❌ Lỗi tải taxi_zones: {e}")
else:
    print("⚡ Taxi taxi_zones đã tồn tại -> Bỏ qua.")

🔄 Đang đồng bộ dữ liệu phụ trợ từ năm 2021 đến 2026...

[2/4] 🚏 Đang tải Taxi taxi_zones (Bản đồ khu vực)...
✅ Đã tải xong Taxi taxi_zones.


In [8]:
import pandas as pd
# Load data into pandas DataFrame from "/lakehouse/default/Files/bronze/taxi_zones/taxi_zone_lookup.csv"
df = pd.read_csv("/lakehouse/default/Files/bronze/taxi_zones/taxi_zone_lookup.csv")
display(df)


# 5 Dataset_3: TẠO DỮ LIỆU NGÀY LỄ (HOLIDAYS)

In [9]:
# ==========================================
# 1. CẤU HÌNH THỜI GIAN ĐỒNG BỘ
# ==========================================
# Phải khớp với khoảng thời gian bạn tải Taxi Data
START_YEAR = 2021
CURRENT_YEAR = datetime.now().year 

# Đường dẫn gốc
base_path = "/lakehouse/default/Files/bronze"
os.makedirs(f"{base_path}/holidays", exist_ok=True)

print(f"🔄 Đang đồng bộ dữ liệu phụ trợ từ năm {START_YEAR} đến {CURRENT_YEAR}...")

# ==========================================
# DATASET 3: US HOLIDAYS (Lịch nghỉ lễ)
# ==========================================
# Dữ liệu ĐỘNG: Cần sinh ra danh sách ngày lễ cho TOÀN BỘ các năm từ 2015-Nay.
print(f"\n[3/4] 📅 Đang tạo dữ liệu ngày lễ ({START_YEAR}-{CURRENT_YEAR})...")

all_holidays = []

# Vòng lặp qua từng năm để lấy ngày lễ
for year in range(START_YEAR, CURRENT_YEAR + 1):
    us_holidays = holidays.US(years=year)
    for date, name in us_holidays.items():
        all_holidays.append({
            "date": date,
            "holiday_name": name,
            "year": year
        })

# Chuyển thành DataFrame và lưu CSV
df_holidays = pd.DataFrame(all_holidays)
df_holidays['date'] = pd.to_datetime(df_holidays['date'])
save_path_holidays = f"{base_path}/holidays/us_holidays_{START_YEAR}_{CURRENT_YEAR}.csv"

df_holidays.to_csv(save_path_holidays, index=False)
print(f"✅ Đã tạo xong file: {save_path_holidays}")
print(f"   Tổng số ngày lễ: {len(df_holidays)}")

🔄 Đang đồng bộ dữ liệu phụ trợ từ năm 2021 đến 2026...

[3/4] 📅 Đang tạo dữ liệu ngày lễ (2021-2026)...
✅ Đã tạo xong file: /lakehouse/default/Files/bronze/holidays/us_holidays_2021_2026.csv
   Tổng số ngày lễ: 75


In [10]:
import pandas as pd
# Load data into pandas DataFrame from "/lakehouse/default/Files/bronze/holidays/us_holidays_2021_2026.csv"
df = pd.read_csv("/lakehouse/default/Files/bronze/holidays/us_holidays_2021_2026.csv")
display(df)


# 6. Dataset_4: WEATHER DATA (Thời tiết)

In [11]:
# ==========================================
# 1. CẤU HÌNH THỜI GIAN ĐỒNG BỘ
# ==========================================
START_YEAR = 2021
CURRENT_YEAR = datetime.now().year 

# Đường dẫn gốc
base_path = "/lakehouse/default/Files/bronze"
os.makedirs(f"{base_path}/weather", exist_ok=True)

print(f"🔄 Đang đồng bộ dữ liệu phụ trợ từ năm {START_YEAR} đến {CURRENT_YEAR}...")

# ==========================================
# DATASET 4: WEATHER DATA (Thời tiết)
# ==========================================
print("\n[4/4] ⛅ Đang tải dữ liệu Thời tiết lịch sử (Central Park)...")

url_weather = "https://raw.githubusercontent.com/toddwschneider/nyc-taxi-data/master/data/central_park_weather.csv"
save_path_weather = f"{base_path}/weather/central_park_weather_full.csv"

try:
    # 1. Tải file gốc về trước
    r = requests.get(url_weather)
    if r.status_code == 200:
        with open(save_path_weather, 'wb') as f:
            f.write(r.content)
        print("✅ Đã tải xong file gốc.")
        
        # 2. Đọc lên để xử lý
        df_weather = pd.read_csv(save_path_weather)
        
        # --- BƯỚC QUAN TRỌNG: CHUẨN HÓA ---
        # Đổi tên cột thành chữ thường và xóa khoảng trắng thừa
        df_weather.columns = df_weather.columns.str.lower().str.strip()
        print(f"   -> Đã đổi tên cột: {list(df_weather.columns)}")
        
        # Chuyển đổi ngày
        df_weather['date'] = pd.to_datetime(df_weather['date'])
        
        # Lọc dữ liệu theo năm
        df_weather = df_weather[
            (df_weather['date'].dt.year >= START_YEAR) & 
            (df_weather['date'].dt.year <= CURRENT_YEAR)
        ]

        min_date = df_weather['date'].min().date()
        max_date = df_weather['date'].max().date()
        
        print(f"   -> Dữ liệu sau khi lọc: {min_date} đến {max_date}")
        
        # 3. LƯU LẠI FILE (Đây là bước code cũ bị thiếu)
        # Ghi đè file cũ bằng dữ liệu đã làm sạch
        df_weather.to_csv(save_path_weather, index=False)
        print("💾 Đã LƯU thành công các thay đổi vào file CSV.")
        
        if min_date.year <= START_YEAR and max_date.year >= (CURRENT_YEAR - 1):
             print("✨ Dữ liệu thời tiết ĐẠT YÊU CẦU đồng bộ!")
        else:
             print("⚠️ Lưu ý: Dữ liệu thời tiết có thể chưa cập nhật đến tháng mới nhất.")
    else:
        print(f"❌ Lỗi tải thời tiết: {r.status_code}")
except Exception as e:
    print(f"❌ Lỗi xử lý thời tiết: {e}")

print("\n🎉 HOÀN TẤT! Bây giờ file trong Lakehouse đã có cột chữ thường.")

🔄 Đang đồng bộ dữ liệu phụ trợ từ năm 2021 đến 2026...

[4/4] ⛅ Đang tải dữ liệu Thời tiết lịch sử (Central Park)...
✅ Đã tải xong file gốc.
   -> Đã đổi tên cột: ['station', 'name', 'date', 'awnd', 'prcp', 'snow', 'snwd', 'tmax', 'tmin']
   -> Dữ liệu sau khi lọc: 2021-01-01 đến 2024-02-29
💾 Đã LƯU thành công các thay đổi vào file CSV.
⚠️ Lưu ý: Dữ liệu thời tiết có thể chưa cập nhật đến tháng mới nhất.

🎉 HOÀN TẤT! Bây giờ file trong Lakehouse đã có cột chữ thường.


In [12]:
import pandas as pd
# Load data into pandas DataFrame from "/lakehouse/default/Files/bronze/weather/central_park_weather_full.csv"
df = pd.read_csv("/lakehouse/default/Files/bronze/weather/central_park_weather_full.csv")
display(df)

# SUMMARY 

In [13]:
import pandas as pd
import os
from pyspark.sql.functions import min, max

# Cấu hình đường dẫn
BASE_PATH = "/lakehouse/default/Files/bronze"

# Danh sách để lưu kết quả
summary_data = []

print("🔄 Đang phân tích dữ liệu để tạo bảng tổng hợp...")

# ====================================================
# 1. NYC TAXI taxi_trips (Big Data)
# ====================================================
try:
    df_taxi_trips = spark.read.parquet(f"{BASE_PATH}/taxi_trips/*.parquet")
    
    # Lấy thống kê (có thể mất 1-2 phút nếu dữ liệu lớn)
    stats = df_taxi_trips.select(
        min("tpep_pickup_datetime").alias("min_date"), 
        max("tpep_pickup_datetime").alias("max_date")
    ).collect()[0]
    
    summary_data.append({
        "Dataset Name": "NYC Taxi taxi_trips",
        "File Type": "Parquet",
        "Total Rows": f"{df_taxi_trips.count():,.0f}",
        "Start Date": str(stats['min_date']),
        "End Date": str(stats['max_date']),
        "Columns": len(df_taxi_trips.columns),
        "Status": "✅ Ready"
    })
except Exception as e:
    summary_data.append({
        "Dataset Name": "NYC Taxi taxi_trips",
        "Status": f"❌ Error: {str(e)[:50]}..."
    })

# ====================================================
# 2. TAXI taxi_zones
# ====================================================
try:
    df_taxi_zones = pd.read_csv(f"{BASE_PATH}/taxi_zones/taxi_zone_lookup.csv")
    summary_data.append({
        "Dataset Name": "Taxi taxi_zones",
        "File Type": "CSV",
        "Total Rows": f"{len(df_taxi_zones):,.0f}",
        "Start Date": "N/A (Static)",
        "End Date": "N/A (Static)",
        "Columns": len(df_taxi_zones.columns),
        "Status": "✅ Ready"
    })
except:
    summary_data.append({"Dataset Name": "Taxi taxi_zones", "Status": "❌ Missing"})

# ====================================================
# 3. WEATHER DATA
# ====================================================
try:
    df_weather = pd.read_csv(f"{BASE_PATH}/weather/central_park_weather_full.csv")
    if 'date' in df_weather.columns:
        df_weather['date'] = pd.to_datetime(df_weather['date'])
        summary_data.append({
            "Dataset Name": "Weather Data",
            "File Type": "CSV",
            "Total Rows": f"{len(df_weather):,.0f}",
            "Start Date": str(df_weather['date'].min().date()),
            "End Date": str(df_weather['date'].max().date()),
            "Columns": len(df_weather.columns),
            "Status": "✅ Ready"
        })
    else:
        summary_data.append({"Dataset Name": "Weather Data", "Status": "⚠️ Column Error"})
except:
    summary_data.append({"Dataset Name": "Weather Data", "Status": "❌ Missing"})

# ====================================================
# 4. HOLIDAYS
# ====================================================
try:
    # Tìm file CSV trong folder holidays
    holiday_files = [f for f in os.listdir(f"{BASE_PATH}/holidays") if f.endswith('.csv')]
    if holiday_files:
        df_holiday = pd.read_csv(f"{BASE_PATH}/holidays/{holiday_files[0]}")
        summary_data.append({
            "Dataset Name": "US Holidays",
            "File Type": "CSV",
            "Total Rows": f"{len(df_holiday):,.0f}",
            "Start Date": f"{df_holiday['year'].min()}-01-01",
            "End Date": f"{df_holiday['year'].max()}-12-31",
            "Columns": len(df_holiday.columns),
            "Status": "✅ Ready"
        })
    else:
        summary_data.append({"Dataset Name": "US Holidays", "Status": "❌ Missing"})
except:
    summary_data.append({"Dataset Name": "US Holidays", "Status": "❌ Error"})

# ====================================================
# HIỂN THỊ KẾT QUẢ
# ====================================================
df_summary = pd.DataFrame(summary_data)

# Sắp xếp lại cột cho đẹp
cols = ["Dataset Name", "Status", "Total Rows", "Start Date", "End Date", "Columns", "File Type"]
df_summary = df_summary[cols]

print("\n📊 BẢNG TỔNG HỢP DỮ LIỆU (DATASET SUMMARY):")
display(df_summary)


🔄 Đang phân tích dữ liệu để tạo bảng tổng hợp...


26/01/09 03:20:41 WARN SQLConf: Cannot load HintErrorHandler org.apache.spark.sql.catalyst.analysis.HintErrorLogger for exception: org.apache.spark.sql.catalyst.analysis.HintErrorLogger.<init>(), callStack: [Ljava.lang.StackTraceElement;@6e0d2d2d, load the default HintErrorLogger
26/01/09 03:20:42 WARN SQLConf: Cannot load HintErrorHandler org.apache.spark.sql.catalyst.analysis.HintErrorLogger for exception: org.apache.spark.sql.catalyst.analysis.HintErrorLogger.<init>(), callStack: [Ljava.lang.StackTraceElement;@41e472e2, load the default HintErrorLogger


# NHẬN XÉT:
📑 BÁO CÁO ĐÁNH GIÁ DỮ LIỆU ĐẦU VÀO (DATA INGESTION PROFILE)  
Dự án: Data Engineering End-to-End trên Microsoft Fabric  
Giai đoạn: Bronze Layer (Ingestion)  
Trạng thái: ✅ Hoàn thành 100%

1. Tổng quan (Executive Summary)
Quy trình thu thập dữ liệu (Ingestion) đã hoàn tất thành công cho 4 bộ dữ liệu mục tiêu. Tất cả các file đã được lưu trữ an toàn trong OneLake (Files/bronze) và sẵn sàng cho các bước xử lý tiếp theo.

- Tổng khối lượng dữ liệu: ~194.5 triệu bản ghi (Big Data Scale).
- Định dạng: Parquet (tối ưu cho Spark) và CSV (cho dữ liệu danh mục).
- Trạng thái toàn hệ thống: ✅ Ready (Sẵn sàng xử lý).


2. Phân tích chi tiết từng bộ dữ liệu

🚖 a. NYC Taxi taxi_trips (Bảng dữ liệu chính)
Loại: Fact Table (Dữ liệu giao dịch).

    Khối lượng: 194,457,948 dòng. Đây là quy mô dữ liệu lớn, rất phù hợp để minh chứng khả năng xử lý của Spark trên Fabric.

    Thời gian (Raw): 2001-01-01 đến 2098-09-11.

    ⚠️ Vấn đề phát hiện: Khoảng thời gian quá rộng so với thực tế.

    Dữ liệu năm 2001: Có thể là lỗi hệ thống cũ hoặc đồng hồ xe sai.

    Dữ liệu năm 2098: Chắc chắn là dữ liệu rác (Outliers).

    Định dạng: Parquet (Chuẩn hóa tốt, đọc nhanh hơn CSV 10-50 lần).

🗺️ b. Taxi taxi_zones (Bảng tham chiếu địa điểm)
Loại: Dimension Table (Dữ liệu tĩnh).

    Số lượng: 265 dòng (Khớp với tiêu chuẩn phân vùng của NYC TLC).

    Trạng thái: Sạch, dùng để map PULocationID và DOLocationID sang tên Quận/Thành phố.

⛅ c. Weather Data (Dữ liệu thời tiết)
    Loại: Dimension Table (Dữ liệu biến đổi theo ngày).

    Thời gian: 2021-01-01 đến 2024-02-29.

    Độ phủ: Hơn 3 năm dữ liệu liên tục.

    Vai trò: Cung cấp các đặc trưng (features) quan trọng như mưa, tuyết để dự đoán nhu cầu đặt xe.

📅 d. US Holidays (Lịch nghỉ lễ)
    Loại: Dimension Table.

    Thời gian: 2021-01-01 đến 2026-12-31.

    Vai trò: Giúp mô hình ML nhận biết các ngày đặc biệt (Lễ Tạ Ơn, Giáng Sinh, Năm Mới) nơi nhu cầu đi lại tăng cao.

3. Đánh giá tính đồng bộ & Chiến lược xử lý (Critical Analysis)

🧩 Vấn đề không khớp thời gian (Time Alignment)
Khi so sánh khoảng thời gian giữa các bộ dữ liệu, ta thấy:

    Taxi Data: 2001 - 2098 (Rất rộng).

    Weather Data: 2021 - 02/2024 (Giới hạn chính).

    Holidays: 2021 - 2026.

👉 KẾT LUẬN QUAN TRỌNG: Để mô hình Machine Learning hoạt động chính xác (cần join đủ 3 bảng), chúng ta chỉ có thể sử dụng dữ liệu trong "Cửa sổ vàng" (Golden Window) là: TỪ 01/2021 ĐẾN 12/2023
